Transfer Learning in DL
- It actually shares the knowledge of previously trained model to new project of same problem. So that time is saved instead of writing it from scratch. Accuracy is increased

In [6]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
batchsize=8

In [8]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

In [9]:
train_data= train_datagen.flow_from_directory(r'C:\Users\Dell\OneDrive\Desktop\SEMESTERS\SEM-6\mini BTP\Prepared Data_1\Train data',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(r'C:\Users\Dell\OneDrive\Desktop\SEMESTERS\SEM-6\mini BTP\Prepared Data_1\Train data',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

Found 64304 images belonging to 2 classes.
Found 16075 images belonging to 2 classes.


In [10]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'C:\Users\Dell\OneDrive\Desktop\SEMESTERS\SEM-6\mini BTP\Prepared Data_1\Test Data',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 4519 images belonging to 2 classes.


In [10]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 80, 80, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 39, 39, 32)           864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 39, 39, 32)           96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 39, 39, 32)           0         ['batch_normalization[0][0

In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [9]:
checkpoint = ModelCheckpoint(r'C:\Users\Dell\OneDrive\Desktop\SEMESTERS\SEM-6\mini BTP\models\model.h5',
                             monitor='val_loss', save_best_only=True, verbose=1)

earlystop = EarlyStopping(monitor='val_loss', patience=25, verbose=1, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1)

callbacks = [checkpoint, earlystop, learning_rate]

In [10]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(train_data, 
                    steps_per_epoch=train_data.samples // batchsize,
                    validation_data=validation_data,
                    validation_steps=validation_data.samples // batchsize,
                    callbacks=callbacks,
                    epochs=50)

C:\Users\Dell\AppData\Local\Temp\ipykernel_21392\3884330983.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,


Epoch 1/50


8038/8038 [==============================] - ETA: 0s - loss: 0.2092 - accuracy: 0.9186
Epoch 1: val_loss improved from inf to 0.20824, saving model to C:\Users\Dell\OneDrive\Desktop\SEMESTERS\SEM-6\mini BTP\models\model.h5


C:\Users\Dell\anaconda3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8038/8038 [==============================] - 943s 117ms/step - loss: 0.2092 - accuracy: 0.9186 - val_loss: 0.2082 - val_accuracy: 0.9092 - lr: 0.0010
Epoch 2/50
8038/8038 [==============================] - ETA: 0s - loss: 0.1790 - accuracy: 0.9308
Epoch 2: val_loss improved from 0.20824 to 0.18108, saving model to C:\Users\Dell\OneDrive\Desktop\SEMESTERS\SEM-6\mini BTP\models\model.h5
8038/8038 [==============================] - 1489s 185ms/step - loss: 0.1790 - accuracy: 0.9308 - val_loss: 0.1811 - val_accuracy: 0.9265 - lr: 0.0010
Epoch 3/50
8038/8038 [==============================] - ETA: 0s - loss: 0.1680 - accuracy: 0.9342
Epoch 3: val_loss improved from 0.18108 to 0.17840, saving model to C:\Users\Dell\OneDrive\Desktop\SEMESTERS\SEM-6\mini BTP\models\model.h5
8038/8038 [==============================] - 1125s 140ms/step - loss: 0.1680 - accuracy: 0.9342 - val_loss: 0.1784 - val_accuracy: 0.9221 - lr: 0.0010
Epoch 4/50
8038/8038 [==============================] - ETA: 0s - loss: 

In [11]:
!pip install pickle-mixin

In [1]:
import pickle

In [13]:
with open('Driver_drowsiness.pkl','wb') as file:
    pickle.dump(model, file)

In [2]:
# Load the model from the file
with open('Driver_drowsiness.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

In [3]:
print(loaded_model)

# Model Evaluation

In [11]:
loss_tr, acc_tr = loaded_model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

C:\Users\Dell\AppData\Local\Temp\ipykernel_14940\4038253159.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  loss_tr, acc_tr = loaded_model.evaluate_generator(train_data)




0.9491633772850037
0.13568949699401855


In [12]:
loss_vr, acc_vr = loaded_model.evaluate_generator(validation_data)
print(loss_vr)
print(acc_vr)

C:\Users\Dell\AppData\Local\Temp\ipykernel_14940\4241953701.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  loss_vr, acc_vr = loaded_model.evaluate_generator(validation_data)


0.1823696345090866
0.9249144792556763


In [13]:
loss_test, acc_test = loaded_model.evaluate_generator(test_data)
print(loss_tr)
print(acc_tr)

C:\Users\Dell\AppData\Local\Temp\ipykernel_14940\564767416.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  loss_test, acc_test = loaded_model.evaluate_generator(test_data)


0.13568949699401855
0.9491633772850037
